<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex07-Ken_French.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investments: Theory and Data Analysis**, Bates, Boyer, and Fletcher

# Examples Chapter 7: The Ken French Data Library
In this example we illustrate how to access data in the Ken French Data Library using a Python function that wraps the library’s HTTP endpoint in an API-style interface.

### Imports and Setup

In [ ]:
# Import packages
%pip install -q simple-finance
import simple_finance as sf
import requests
import pandas as pd
import numpy as np

# Establish display options when printing DataFrames
pd.set_option('display.max_columns', None)   # Show all columns without truncation
pd.set_option('display.width', 1000)   # Set the display width so output stays on one line
pd.set_option("display.max_rows", 20) # Force truncation if DataFrame has more than 20 rows

### Load in Data Using the Python Function

####Function: `get_ff_strategies()`

**Inputs**  
* `stype`: type of sstrategy. This input can be one of five strings:  
  - [`beta`](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_port_form_BETA.html)  
  - [`momentum`](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_mom_factor.html)  
  - [`shortermreversal`](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_st_rev_factor.html)
  - [`accruals`](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_port_form_AC.html)  
  - [`value`](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_form_btm.html)  
By clicking on each link you can learn more about each sorting strategy.
* `start_date` (*optional*): the starting date for the data entered as a string: `yyyy-mm`. If not included the data will begin on the earliest possible date.
* `end_date` (*optional*): the ending date for the data entered as a string `yyyy-mm`.  If not included the data will end on the latest possible date.
* `details` (*optional*): If `True`, prints descriptive information about the trading strategy to the terminal. Default is `False`.
* `factors` (*optional*): If equal to the string `FF5` will include all factors from the Fama-French Five factor model, and the return on short-term t-bills. Otherwise, default is to include all factors from the Fama-French Three factor model, and the return on short-term t-bills.


**Output**  
DataFrame
* Decile portfolios of the chosen strategy.
* Factor returns
* Return on short-term T-bills.

** Example **  
`df=get_ff_strategies('momentum',start_date='2020-01', details=True)`  
This call returns decile portfolio returns for the momentum strategy beginning in January 2020 and continuing to the present, and prints descriptive information about the strategy to the terminal.


In [ ]:
# Get returns on momentum deciles
df=sf.get_ff_strategies('shorttermreversal', details=True)

### Get summary statistics

In [ ]:
# Define decile columns
DECILES = [
    "Dec 1", "Dec 2", "Dec 3", "Dec 4", "Dec 5",
    "Dec 6", "Dec 7", "Dec 8", "Dec 9", "Dec 10"
]

# Define excess returns
excess = df[DECILES].sub(df["rf"], axis=0)

# create DataFrame of summary statistics
stats = pd.DataFrame({
    "Mean": df[DECILES].mean(),
    "Std": df[DECILES].std(),
    "Sharpe": excess.mean() / excess.std(),
    "Min": df[DECILES].min(),
    "Max": df[DECILES].max(),
    "N": df[DECILES].count()
})

# Annualize
stats["Ann Mean"] = 12 * stats["Mean"]
stats["Ann Std"]  = np.sqrt(12) * stats["Std"]
stats["Ann Sharpe"] = stats['Sharpe'] * np.sqrt(12)

print(stats)

### Test Trading Strategy

In [ ]:
# Create realized portfolio returns
# weights: market 1.0, decile 1 0.50, decile 10 -0.50
rp = (df["mkt-rf"] + df["rf"]+ 0.50 * df["Dec 1"] - 0.50 * df["Dec 10"])
excess =(df["mkt-rf"] + 0.50 * df["Dec 1"] - 0.50 * df["Dec 10"])

# Get monthly mean returns, volatility, and sharpe
mean_monthly = rp.mean()
std_monthly  = rp.std()
mean_excess_monthly  = excess.mean()
sharpe_monthly = mean_excess_monthly / std_monthly

# annualize
ann_return = 12 * mean_monthly
ann_vol    = np.sqrt(12) * std_monthly
ann_sharpe = sharpe_monthly * np.sqrt(12)

# Print results
print(f"Average Return: {ann_return:.3f}")
print(f"Annualized Volatility: {ann_vol:.3f}")
print(f"Annualized Sharpe Ratio: {ann_sharpe:.3f}")
